## Dropout
Dropout是防止overfitting的一种方式，在训练过程中会随机的让网络某些隐含层节点的权重不工作，不工作的那些节点可以暂时认为不是网络结构的一部分，但是它的权重得保留下来（只是暂时不更新而已），因为下次样本输入时它可能又得工作了。我没有找到dropout的工作原理的数学解释。但是实验表明，很有用，这个例子展示了如何用tensorflow来实现dropout功能。因为我们需要自己定义一个权重使用的概率（在训练过程中让多少权重工作或者不工作的百分比。），所以我们要比之前的例子多增加一个placeholder。

我们依然可以通过tensorboard来观看模型训练的情况。

In [1]:
from __future__ import print_function
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
# 导入数据同时将数据分成两部分：训练集（0.7），测试集（0.3）
# 我们使用one hot形式来表示y。
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)


In [3]:
def add_layer(inputs, in_size, out_size, layer_name, activation_function=None, ):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    # 这个地方我们实现了dropout。
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b, )
    #tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

In [4]:
def compute_accuracy(v_xs, v_ys):
    ''' 
    计算精度的函数，prediction里的元素是一个10维的向量，其中最大值的indice就对应着预测的数字。
    '''
    global prediction
    #注意我们在预测的时候就不能使用dropout，所以把keep_prob定为1
    y_pre = sess.run(prediction, feed_dict={xs: v_xs,keep_prob: 1.0})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1.0})
    return result

In [5]:
# 定义网络输入的 placeholder
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 64])  # 8x8
ys = tf.placeholder(tf.float32, [None, 10])

# 增加一个隐藏层
l1 = add_layer(xs, 64, 50, 'l1', activation_function=tf.nn.tanh)
# 增加输出层
prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)

# 损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1])) 
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()

if int((tf.__version__).split('.')[1]) < 12:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

In [6]:
# 训练
for i in range(500):
    sess.run(train_step, feed_dict={xs: X_train, ys: y_train, keep_prob: 0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            X_test, y_test))


0.177778
0.705556
0.803704
0.831481
0.844444
0.857407
0.868519
0.885185
0.888889
0.87963
